# Análisis de Calidad de Datos - Detección de Títulos Problemáticos

Después del procesamiento completo, vamos a analizar la calidad de los datos para detectar películas con títulos problemáticos en la tabla `peliculas`.

**Estructura de la base de datos:**
- Tabla `peliculas`: campo `titulo` definido como `TEXT NOT NULL`
- Tabla `peliculas_generos`: relación M:M
- Clave primaria: `movie_id`

**Tipos de problemas a detectar:**
- Títulos con solo espacios en blanco `" "`, `"  "`, `"   "`
- Títulos muy cortos (≤ 2 caracteres después de TRIM)
- Títulos con caracteres especiales

**Método de limpieza:**
Eliminación en dos pasos (primero `peliculas_generos`, después `peliculas`)

In [ ]:
import boto3
import json
import os
from dotenv import load_dotenv
from datetime import datetime
import psycopg
import pandas as pd

# Cargar variables de entorno
load_dotenv()

In [ ]:
# CONFIGURACIÓN PARA ANÁLISIS DE CALIDAD DE DATOS

# Recuperar credenciales desde Secrets Manager
db_secret_name = os.getenv('DB_KEY')
aws_region = os.getenv('REGION')

if not db_secret_name or not aws_region:
    print("ERROR: Variables DB_KEY y/o REGION no configuradas")
    exit()

try:
    # Crear cliente de Secrets Manager y obtener credenciales
    secrets_client = boto3.client('secretsmanager', region_name=aws_region)
    response = secrets_client.get_secret_value(SecretId=db_secret_name)
    secret_data = json.loads(response['SecretString'])
    
    # Configurar conexión a la base de datos
    DB_CONFIG = {
        'host': secret_data.get('host', ''),
        'dbname': secret_data.get('dbname', ''),
        'user': secret_data.get('username', ''),
        'password': secret_data.get('password', ''),
        'port': secret_data.get('port', 5432)
    }
    
    def execute_query(query, description=""):
        # Ejecuta una consulta SQL y retorna los resultados como DataFrame
        try:
            with psycopg.connect(**DB_CONFIG) as conn:
                df = pd.read_sql_query(query, conn)
                return df
        except Exception as e:
            print(f"Error ejecutando consulta: {str(e)}")
            return None
    
    print("Configuración completada correctamente")
    
except Exception as e:
    print(f"ERROR: {str(e)}")

In [ ]:
# DETECCIÓN DE TÍTULOS PROBLEMÁTICOS
print("ANÁLISIS DE CALIDAD TÍTULOS")
print("=" * 30)

try:
    # Contar total de películas
    query_total = "SELECT COUNT(*) as total_peliculas FROM peliculas;"
    df_total = execute_query(query_total)
    if df_total is not None:
        total_peliculas = df_total['total_peliculas'].iloc[0]
        print(f"Total de películas: {total_peliculas:,}")

    # Detectar títulos problemáticos
    query_problematicos = """
    SELECT COUNT(*) as titulos_problematicos 
    FROM peliculas 
    WHERE titulo IN (' ', '  ', '   ') 
       OR LENGTH(TRIM(titulo)) <= 2
       OR TRIM(titulo) = ''
       OR titulo ~ '^[^a-zA-Z0-9]+$';
    """
    df_problematicos = execute_query(query_problematicos)
    if df_problematicos is not None:
        titulos_problematicos = df_problematicos['titulos_problematicos'].iloc[0]
        print(f"Títulos problemáticos: {titulos_problematicos:,}")
        
        if total_peliculas > 0:
            porcentaje_problematicos = (titulos_problematicos / total_peliculas) * 100
            print(f"Porcentaje a eliminar: {porcentaje_problematicos:.1f}%")

except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
# CONFIRMACIÓN DE TÍTULOS A ELIMINAR
print("CONFIRMACIÓN DE TÍTULOS A ELIMINAR")
print("=" * 35)

try:
    # Contar total exacto de películas que se eliminarán
    query_total_eliminar = """
    SELECT COUNT(*) as total_a_eliminar
    FROM peliculas 
    WHERE titulo IN (' ', '  ', '   ') 
       OR LENGTH(TRIM(titulo)) <= 2
       OR TRIM(titulo) = ''
       OR titulo ~ '^[^a-zA-Z0-9]+$';
    """
    
    df_total_eliminar = execute_query(query_total_eliminar)
    if df_total_eliminar is not None:
        total_a_eliminar = df_total_eliminar['total_a_eliminar'].iloc[0]
        print(f"Películas marcadas para eliminar: {total_a_eliminar:,}")
        
        if total_a_eliminar > 0:
            # Calcular porcentaje sobre el total
            if 'total_peliculas' in locals():
                porcentaje_eliminacion = (total_a_eliminar / total_peliculas) * 100
                print(f"Porcentaje de eliminación: {porcentaje_eliminacion:.2f}%")
            print("Listo para proceder con la eliminación")
        else:
            print("No hay títulos problemáticos para eliminar")

except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
# PREPARACIÓN PARA LIMPIEZA
print("PREPARACIÓN PARA LIMPIEZA")
print("=" * 30)

# Condición para títulos problemáticos
problematic_condition = """titulo IN (' ', '  ', '   ') 
       OR LENGTH(TRIM(titulo)) <= 2
       OR TRIM(titulo) = ''
       OR titulo ~ '^[^a-zA-Z0-9]+$'"""

# Script SQL para eliminación en dos pasos - guardado en variables
cleanup_sql_manual = f"""
DELETE FROM peliculas_generos 
WHERE movie_id IN (
    SELECT movie_id
    FROM peliculas 
    WHERE {problematic_condition}
);

DELETE FROM peliculas 
WHERE {problematic_condition};
"""

# Script de verificación
verification_sql = """
SELECT 
    'Después de eliminación' as estado,
    COUNT(*) as total,
    COUNT(CASE WHEN LENGTH(TRIM(titulo)) > 2 
              AND titulo NOT IN (' ', '  ', '   ')
              AND TRIM(titulo) != ''
              AND titulo !~ '^[^a-zA-Z0-9]+$' THEN 1 END) as titulos_validos,
    ROUND(
        (COUNT(CASE WHEN LENGTH(TRIM(titulo)) > 2 
                    AND titulo NOT IN (' ', '  ', '   ')
                    AND TRIM(titulo) != ''
                    AND titulo !~ '^[^a-zA-Z0-9]+$' THEN 1 END) * 100.0 / COUNT(*)), 2
    ) as porcentaje_calidad
FROM peliculas;
"""

print("Scripts SQL preparados para eliminación manual")
print("Ejecuta la siguiente celda para proceder con la eliminación")

In [ ]:
# ELIMINAR FILAS CON TÍTULOS PROBLEMÁTICOS
print("ELIMINANDO FILAS CON TÍTULOS PROBLEMÁTICOS")
print("=" * 50)

try:
    # 1. Contar registros antes de la eliminación
    query_antes = "SELECT COUNT(*) as total_antes FROM peliculas;"
    df_antes = execute_query(query_antes)
    total_antes = df_antes['total_antes'].iloc[0] if df_antes is not None else 0
    
    print(f"Películas antes de limpieza: {total_antes:,}")
    
    # 2. Identificar películas problemáticas
    identify_script = f"""
    SELECT COUNT(*) as problematicas
    FROM peliculas 
    WHERE {problematic_condition};
    """
    
    df_problematicas = execute_query(identify_script)
    problematicas_count = 0
    if df_problematicas is not None:
        problematicas_count = df_problematicas['problematicas'].iloc[0]
        print(f"Películas detectadas para eliminar: {problematicas_count:,}")
    
    if problematicas_count > 0:
        # 3. Eliminación manual en 2 pasos
        movies_deleted = 0
        genres_deleted = 0
        
        print("Ejecutando eliminación en dos pasos...")
        
        with psycopg.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                try:
                    # Paso 1: eliminar referencias en peliculas_generos
                    cur.execute(f"""
                    DELETE FROM peliculas_generos 
                    WHERE movie_id IN (
                        SELECT movie_id
                        FROM peliculas 
                        WHERE {problematic_condition}
                    );
                    """)
                    genres_deleted = cur.rowcount
                    
                    # Paso 2: eliminar películas problemáticas
                    cur.execute(f"""
                    DELETE FROM peliculas 
                    WHERE {problematic_condition};
                    """)
                    movies_deleted = cur.rowcount
                    
                    # Confirmar cambios
                    conn.commit()
                    print("Eliminación completada exitosamente")
                    
                except Exception as e:
                    conn.rollback()
                    raise Exception(f"Error durante la eliminación: {str(e)}")
        
        # 4. Verificar resultados
        verification_script = """
        SELECT 
            COUNT(*) as total,
            COUNT(CASE WHEN titulo IN (' ', '  ', '   ') 
                      OR LENGTH(TRIM(titulo)) <= 2
                      OR TRIM(titulo) = ''
                      OR titulo ~ '^[^a-zA-Z0-9]+$' THEN 1 END) as titulos_problematicos,
            ROUND(
                (COUNT(CASE WHEN LENGTH(TRIM(titulo)) > 2 
                            AND titulo NOT IN (' ', '  ', '   ')
                            AND TRIM(titulo) != ''
                            AND titulo !~ '^[^a-zA-Z0-9]+$' THEN 1 END) * 100.0 / COUNT(*)), 2
            ) as porcentaje_calidad
        FROM peliculas;
        """
        
        df_verificacion = execute_query(verification_script)
        
        if df_verificacion is not None:
            row = df_verificacion.iloc[0]
            total_despues = row['total']
            
            print(f"\nRESULTADOS:")
            print(f"Películas eliminadas: {movies_deleted:,}")
            print(f"Relaciones género eliminadas: {genres_deleted:,}")
            print(f"Total después: {total_despues:,}")
            print(f"Problemas restantes: {row['titulos_problematicos']:,}")
            print(f"Calidad final: {row['porcentaje_calidad']}%")
            
            if row['porcentaje_calidad'] == 100.0:
                print("CALIDAD PERFECTA")
            elif row['porcentaje_calidad'] >= 99.0:
                print("CALIDAD EXCELENTE")
                
    else:
        print("No se encontraron películas problemáticas para eliminar")
        print("La base de datos ya tiene calidad excelente")

except Exception as e:
    print(f"Error durante la eliminación: {str(e)}")

In [ ]:
# REPORTE FINAL
print("REPORTE FINAL")
print("=" * 15)

try:
    # Estado final de calidad de títulos
    query_calidad_final = """
    SELECT 
        COUNT(*) as total,
        COUNT(CASE WHEN titulo IN (' ', '  ', '   ') 
                  OR LENGTH(TRIM(titulo)) <= 2
                  OR TRIM(titulo) = ''
                  OR titulo ~ '^[^a-zA-Z0-9]+$' THEN 1 END) as problematicos,
        ROUND(
            (COUNT(CASE WHEN LENGTH(TRIM(titulo)) > 2 
                        AND titulo NOT IN (' ', '  ', '   ')
                        AND TRIM(titulo) != ''
                        AND titulo !~ '^[^a-zA-Z0-9]+$' THEN 1 END) * 100.0 / COUNT(*)), 2
        ) as porcentaje_calidad
    FROM peliculas;
    """
    
    df_calidad = execute_query(query_calidad_final)
    if df_calidad is not None:
        calidad = df_calidad.iloc[0]
        print(f"Total películas: {calidad['total']:,}")
        print(f"Títulos problemáticos: {calidad['problematicos']:,}")
        print(f"Calidad: {calidad['porcentaje_calidad']}%")

        if calidad['porcentaje_calidad'] == 100.0:
            print("BASE DE DATOS LISTA")
        elif calidad['porcentaje_calidad'] >= 99.0:
            print("CALIDAD EXCELENTE")
        else:
            print("REVISAR CALIDAD")

except Exception as e:
    print(f"Error: {str(e)}")